Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [670]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [671]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [672]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [673]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [674]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])
# first_letter = int representing lowercase "a"
#
# ord(c):
# Given a string of length one, return an integer representing 
# the Unicode code point of the character when the argument is 
# a unicode object, or the value of the byte when the argument 
# is an 8-bit string. For example, ord('a') returns the 
# integer 97, ord(u'\u2020') returns 8224. This is the inverse 
# of chr() for 8-bit strings and of unichr() for unicode 
# objects. If a unicode argument is given and Python was built 
# with UCS2 Unicode, then the character’s code point must be 
# in the range [0..65535] inclusive; otherwise the string 
# length is two, and a TypeError will be raised.


def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [675]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """
     Generate a single batch from the current cursor position in the data.
     Note that this takes a single character from each segment (where
     segment size = self._text_size // batch_size = 1,562,484 - so a
     single character every 1,562,484 characters), and only makes sense 
     in the context of self.next() below, which forms the 64 11-char 
     strings in the batch.
    """
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    # loop over single character in 1,562,484-character segment
    for b in range(self._batch_size):
      # 1-hot encoding of single character within segment b 
      # (starting at self._cursor[b])
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      # update self._cursor[b] for following loop within self.next()
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """
      Generate the next array of batches from the data. The array consists of
      the last batch of the previous array, followed by num_unrollings new ones.
      
    """
    batches = [self._last_batch]
    # self._num_unrollings + 1 = length of strings in batch
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [676]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  # make 1e-10 lowest possible value for predictions
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """
    Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    Increased prob. that higher probabilities will cause s>=r.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  # prediction[0] = actual list of (probability distribution) predictions
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [330]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  # gradient clipping
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [332]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.294512 learning rate: 10.000000
Minibatch perplexity: 26.96
nltcsmbge oyrgec wufralbpr rseaicnhn fegimvhitn itepvo tienadcllmoste cg lsuzzsl
bwmcifen ozxo envn ofbsratiejgymnedjtteekygzxefytbue sej gra ld sao oln xivvgayx
h ia  eonaahdfarl nt ze deww e hhin lboc vh r knbxjeh uoq  zf   mb w gzozwbcakmn
e zuyet agynaghvkcpqtfdgouernkuqomsoj kxv aianeaps replejhn gsbawjl ayka pmvae e
ftw a teakmru agsaoinkkwxhqnb geckuwn semuen brqq mzazlhlpej kelnpveapitiirrepem
Validation set perplexity: 20.30
Average loss at step 100: 2.611465 learning rate: 10.000000
Minibatch perplexity: 10.26
Validation set perplexity: 10.15
Average loss at step 200: 2.270834 learning rate: 10.000000
Minibatch perplexity: 8.50
Validation set perplexity: 9.24
Average loss at step 300: 2.076317 learning rate: 10.000000
Minibatch perplexity: 7.64
Validation set perplexity: 7.92
Average loss at step 400: 1.996258 learning rate: 10.000000
Minibatch perplexity: 6.55
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [333]:
# lstm_cell() redefined

num_nodes = 64

graph = tf.Graph()
with graph.as_default():

  # Parameters:
  # all gates: input, previous output, and bias.
  simple_x = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  simple_m = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  simple_b = tf.Variable(tf.zeros([1, 4*num_nodes]))
  # input, forget, memory, output range dividers
  div_if = num_nodes
  div_fc = num_nodes*2
  div_co = num_nodes*3
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    simplified = tf.matmul(i, simple_x) + tf.matmul(o, simple_m) + simple_b
    input_gate = tf.sigmoid(simplified[:, :div_if])
    forget_gate = tf.sigmoid(simplified[:, div_if:div_fc])
    update = simplified[:, div_fc:div_co]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(simplified[:, div_co:])
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  # gradient clipping
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [334]:
# same as above

num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.296916 learning rate: 10.000000
Minibatch perplexity: 27.03
vkzyeaqbfzckiunaa kh emzp  a   a  llftec vdphpi pfpdenam odcxireed naiqic e  rsn
u k vtklfludibrctemtt tu  acljsbe nma rril zjdatqsi ji okvrxrs edmvdno fisagapor
bapzlhqt  rki abel btqlpeshidqx eoaomeri t  wii meegshe hvehrletyquvwhrzpaaaj rl
f keyrbj ha reqllneegboxdypmodesle hpxeaoxeftyewrqrx wiux  ko vskf h rp vpaf  ow
zqrlt  zwfi  uluxrosvoielfpxbatasuem tmrz jmpeeijgkvwncrawr   s aeueqckejaedg un
Validation set perplexity: 20.35
Average loss at step 100: 2.609390 learning rate: 10.000000
Minibatch perplexity: 10.75
Validation set perplexity: 10.29
Average loss at step 200: 2.292040 learning rate: 10.000000
Minibatch perplexity: 8.39
Validation set perplexity: 9.32
Average loss at step 300: 2.114084 learning rate: 10.000000
Minibatch perplexity: 7.65
Validation set perplexity: 8.73
Average loss at step 400: 2.036924 learning rate: 10.000000
Minibatch perplexity: 7.05
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

Code below based on approach used by github.com/vrasneur/udacity-deep_learning/blob/master/6_lstm.ipynb

---

__a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.__

---

In [527]:
class BatchGeneratorEmbed(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """
     Generate a single batch from the current cursor position in the data.
     Note that this takes a single character from each segment (where
     segment size = self._text_size // batch_size = 1,562,484 - so a
     single character every 1,562,484 characters), and only makes sense 
     in the context of self.next() below, which forms the 64 11-char 
     strings in the batch.
    """
    batch = np.zeros(shape=(self._batch_size), dtype=np.float)
    # loop over single character in 1,562,484-character segment
    for b in range(self._batch_size):
      # differs from BatchGenerator() in outputing character ID
      # not one-hot encoding
      batch[b] = char2id(self._text[self._cursor[b]])
      # update self._cursor[b] for following loop within self.next()
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """
      Generate the next array of batches from the data. The array consists of
      the last batch of the previous array, followed by num_unrollings new ones.
      
    """
    batches = [self._last_batch]
    # self._num_unrollings + 1 = length of strings in batch
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

embed_train_batches = BatchGeneratorEmbed(train_text, batch_size, num_unrollings)
embed_valid_batches = BatchGeneratorEmbed(valid_text, 1, 1)

In [528]:
def embed_characters(ids):
  """Turn a character IDs into their character representation."""
  return [id2char(int(c)) for c in ids]

def embed_batches2string(batches):
  """Convert a sequence of batches back into their string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

def embed_sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def embed_random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [529]:
graph = tf.Graph()
with graph.as_default():

  # Parameters:
  # Input gate: input, previous output, and bias.
  simple_x = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  simple_m = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  simple_b = tf.Variable(tf.zeros([1, 4*num_nodes]))
  # inpput, forget, memory, output range dividers
  div_if = num_nodes
  div_fc = num_nodes*2
  div_co = num_nodes*3
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    # core of embedding approach
    simplified = tf.nn.embedding_lookup(simple_x, i) + tf.matmul(o, simple_m) + simple_b
    
    input_gate = tf.sigmoid(simplified[:, :div_if])
    forget_gate = tf.sigmoid(simplified[:, div_if:div_fc])
    update = simplified[:, div_fc:div_co]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(simplified[:, div_co:])
    
    return output_gate * tf.tanh(state), state


  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int64, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  # gradient clipping
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


In [536]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = embed_train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])

      # convert from char ID to to one-hot encoding
      # and use those to calculate perplexity
      noembed_labels = np.zeros(predictions.shape)
      for i, j in enumerate(labels):
        noembed_labels[i, int(j)] = 1.0

      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, noembed_labels))))
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
          feed = embed_sample(embed_random_distribution())

          # convert from one-hot encoding to char ID
          feed = [np.argmax(feed)]

          sentence = embed_characters(feed)[0]
          reset_sample_state.run()

          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = embed_sample(prediction)

            # convert from one-hot encoding to char ID
            feed = [np.argmax(feed)]

            sentence += embed_characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = embed_valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})

        # convert char ID to one-hot encoding
        onehot_char = np.zeros((1, vocabulary_size))
        onehot_char[0, int(b[1])] = 1.0

        valid_logprob = valid_logprob + logprob(predictions, onehot_char)
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295797 learning rate: 10.000000
Minibatch perplexity: 27.00
xqrcpp saafcykhwuxahimeg   ooslmgbgr nmnkdkzc eufhwiavoibjioot grjeidtcirqqvq by
waufh mke nkxq ucpeg avrnsu deokipbos j hk fc pg a wcpxoyysqcfgvftl fosat jbdsf 
s jgtksct veil hsmh oloxtnjgr rki sqkm g ufcmysmdnio znzxngog stgod r oezobtryb 
zsx oa tdtg entm raagcf nvbklfuhuxvyugehcss clgghl cyyvgyiid s smjwcei fcwft fli
yv nsfwng thh jinrrcnsetfbnrwto oadieyeeuogmwvfat h ogaamm aiaenzc toz dizfdvyrg
Validation set perplexity: 20.05
Average loss at step 100: 2.565100 learning rate: 10.000000
Minibatch perplexity: 10.54
Validation set perplexity: 11.41
Average loss at step 200: 2.242946 learning rate: 10.000000
Minibatch perplexity: 8.66
Validation set perplexity: 9.37
Average loss at step 300: 2.094970 learning rate: 10.000000
Minibatch perplexity: 7.95
Validation set perplexity: 8.35
Average loss at step 400: 2.045563 learning rate: 10.000000
Minibatch perplexity: 8.04
Validation set per

---

__b- Write a bigram-based LSTM, modeled on the character LSTM above.__

---

In [677]:
bigram_vocabulary_size = vocabulary_size**2

class BatchGeneratorBigram(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size), dtype=np.float)
    for b in range(self._batch_size):
      first_char = self._text[self._cursor[b]]
      if self._cursor[b] + 1 == self._text_size:
        second_char = ' '
      else:
        second_char = self._text[self._cursor[b] + 1]
      batch[b] = char2id(first_char) * vocabulary_size + char2id(second_char)
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
    
  def next(self):
    """
      Generate the next array of batches from the data. The array consists of
      the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    # self._num_unrollings + 1 = length of strings in batch
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

bigram_train_batches = BatchGeneratorBigram(train_text, batch_size, num_unrollings)
bigram_valid_batches = BatchGeneratorBigram(valid_text, 1, 1)

In [678]:
def bigram_id2char(bigram_id):
    c1 = id2char(int((bigram_id//vocabulary_size)))
    c2 = id2char(int(bigram_id%vocabulary_size))
    return ''.join([c1, c2])

def bigram_characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return ['{:s}{:s}'.format(id2char(int(c)//vocabulary_size), id2char(int(c)%vocabulary_size)) 
           for c in np.argmax(probabilities, 1)]

def bigram_first_character(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its first character."""
  return [id2char(c//vocabulary_size) for c in np.argmax(probabilities, 1)]

def bigram_batches2string(batches):
  """Convert a sequence of batches back into their string representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    chars_b = list()
    for bigram_id in b:
      chars_b.append(bigram_id2char(bigram_id))
    s = [''.join(x) for x in zip(s, chars_b)]
  return s

def bigram_sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, bigram_vocabulary_size], dtype=np.float)
  # prediction[0] = actual list of (probability distribution) predictions
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def bigram_random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, bigram_vocabulary_size])
  return b/np.sum(b, 1)[:,None]

def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  # make 1e-10 lowest possible value for predictions
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / len(labels)

print(bigram_batches2string(bigram_train_batches.next()))
print(bigram_batches2string(bigram_train_batches.next()))
print(bigram_batches2string(bigram_valid_batches.next()))
print(bigram_batches2string(bigram_valid_batches.next()))

['onnss  aannaarrcchhiis', 'whheenn  mmiilliittaar', 'llleerriiaa  aarrcchhe', ' aabbbbeeyyss  aanndd ', 'maarrrriieedd  uurrrra', 'heell  aanndd  rriicch', 'y  aanndd  lliittuurrg', 'ayy  ooppeenneedd  ffo', 'tiioonn  ffrroomm  tth', 'miiggrraattiioonn  tto', 'neeww  yyoorrkk  ootth', 'hee  bbooeeiinngg  sse', 'e  lliisstteedd  wwiit', 'ebbeerr  hhaass  pprro', 'o  bbee  mmaaddee  tto', 'yeerr  wwhhoo  rreecce', 'orree  ssiiggnniiffiic', 'a  ffiieerrccee  ccrri', ' ttwwoo  ssiixx  eeiig', 'arriissttoottllee  ss ', 'ittyy  ccaann  bbee  l', ' aanndd  iinnttrraacce', 'tiioonn  ooff  tthhee ', 'dyy  ttoo  ppaassss  h', 'f  cceerrttaaiinn  ddr', 'att  iitt  wwiillll  t', 'e  ccoonnvviinnccee  t', 'enntt  ttoolldd  hhiim', 'ammppaaiiggnn  aanndd ', 'rvveerr  ssiiddee  sst', 'ioouuss  tteexxttss  s', 'o  ccaappiittaalliizze', 'a  dduupplliiccaattee ', 'ghh  aannnn  eess  dd ', 'innee  jjaannuuaarryy ', 'roossss  zzeerroo  tth', 'caall  tthheeoorriiees', 'asstt  iinnssttaanncce', ' ddiimmeen

In [636]:
graph = tf.Graph()
with graph.as_default():

  # Parameters:
  # Input gate: input, previous output, and bias.
  simple_x = tf.Variable(tf.truncated_normal([bigram_vocabulary_size, 4*num_nodes], -0.1, 0.1))
  simple_m = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  simple_b = tf.Variable(tf.zeros([1, 4*num_nodes]))
  # inpput, forget, memory, output range dividers
  div_if = num_nodes
  div_fc = num_nodes*2
  div_co = num_nodes*3
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    # core of embedding approach
    simplified = tf.nn.embedding_lookup(simple_x, i) + tf.matmul(o, simple_m) + simple_b
    input_gate = tf.sigmoid(simplified[:, :div_if])
    forget_gate = tf.sigmoid(simplified[:, div_if:div_fc])
    update = simplified[:, div_fc:div_co]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(simplified[:, div_co:])

    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int64, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  # gradient clipping
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [637]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = bigram_train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])

      # convert from char ID to to one-hot encoding
      # and use those to calculate perplexity
      noembed_labels = np.zeros(predictions.shape)
      for i, j in enumerate(labels):
        noembed_labels[i, int(j)] = 1.0

      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, noembed_labels))))
      
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):

          # random start sample (in 1-hot encoding)
          feed = bigram_sample(bigram_random_distribution())

          # start sentence (in characters)
          # e.g. bigram_characters(feed) = ['ft']
          # convert from one-hot encoding to bigram ID
          
          bigram_sentence = bigram_characters(feed)

          # start sentence (in characters)
          # e.g. bigram_first_character(feed) = ['f']
          sentence = bigram_first_character(feed)[0]
          
          feed = [np.argmax(feed)]
          reset_sample_state.run()

          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = bigram_sample(prediction)
            bigram_sentence += bigram_characters(feed)
            sentence += bigram_first_character(feed)[0]
            feed = [np.argmax(feed)]
          print('bigrams:', bigram_sentence)
          print('chars:', sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = bigram_valid_batches.next()
        predictions = sample_prediction.eval({sample_input: [b[0][0]]})

        # convert char ID to one-hot encoding
        onehot_char = np.zeros((1, bigram_vocabulary_size))
        onehot_char[0, int(b[1][0])] = 1.0

        valid_logprob = valid_logprob + logprob(predictions, onehot_char)
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))


Initialized
Average loss at step 0: 6.591292 learning rate: 10.000000
Minibatch perplexity: 728.72
bigrams: ['ru', 'rn', 'rq', 'lj', 'pq', 'da', 'nt', ' t', 'vr', 'bj', ' w', 'rq', 'jt', 'kw', 'zj', 'rv', 'pu', 'tq', 'bd', 'gn', 'cd', 'jn', 'gk', ' y', 'dy', 'pl', 'qc', 'pa', 'c ', 'jr', 'aq', 'mo', 'ub', 'kr', 'ue', 'vh', 'sm', 'c ', 'sr', 'hx', 'uf', 'ip', 'gw', 'ft', 'ys', 'si', 'po', 'gn', 'gq', 'vo', 'yp', 'ow', 'bs', 'at', 'yh', 'zv', 'bv', 'fp', 'rn', 'ny', 'ej', 'pr', 'ih', 'ri', 'ml', 'bw', 'gi', 'tw', 'tn', 'uq', 'nr', 'fh', 'jw', 'zv', 'ol', 'my', 'qt', 'w ', 'gi', 'bf']
chars: rrrlpdn vb rjkzrptbgcjg dpqpcjamukuvscshuigfyspggvyobayzbfrnepirmbgttunfjzomqwgb
bigrams: ['eb', 'hl', 'fh', 'qa', 'dl', 'el', 'qe', 'rl', 'qm', 'ye', 'lx', 'jr', 'bd', 'vx', 'nq', 'gt', 'on', 'vi', ' w', 'kz', 'yy', 'xz', 'lj', 'vz', 'ai', 'fc', 'xo', 'i ', 'op', 'ih', 'tt', 'pp', 'lt', 'dw', 'zj', 'we', 'ki', 'yx', 'wx', 'dp', 'uu', 'gl', 'dj', 'mf', 'fj', 'vb', 'iy', 'gi', 'kc', 'hn', 'rw', 'ih', '

---

__c- Introduce Dropout.__

---

Rough grid search

In [ ]:
keep_rates = [0.95, 0.875, 0.75, 0.5, 0.25]
keep_rate_sum = [0.]*len(keep_rates)

for kr_loop_counter in xrange(10):

  for kridx, kr in enumerate(keep_rates):

    print("="*80 + "\n" + "="*80)
    print("loop", kr_loop_counter)
    print("kr", kr)

    graph = tf.Graph()
    with graph.as_default():

      # Parameters:
      # Input gate: input, previous output, and bias.
      simple_x = tf.Variable(tf.truncated_normal([bigram_vocabulary_size, 4*num_nodes], -0.1, 0.1))
      simple_m = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
      simple_b = tf.Variable(tf.zeros([1, 4*num_nodes]))
      # inpput, forget, memory, output range dividers
      div_if = num_nodes
      div_fc = num_nodes*2
      div_co = num_nodes*3
      # Variables saving state across unrollings.
      saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      # Classifier weights and biases.
      w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
      b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
      
      # Definition of the cell computation.
      def lstm_cell(i, o, state, train=False):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        # core of embedding approach
        if train:
          simplified = tf.nn.dropout(tf.nn.embedding_lookup(simple_x, i), kr) + tf.matmul(o, simple_m) + simple_b
        else:
          simplified = tf.nn.embedding_lookup(simple_x, i) + tf.matmul(o, simple_m) + simple_b
        input_gate = tf.sigmoid(simplified[:, :div_if])
        forget_gate = tf.sigmoid(simplified[:, div_if:div_fc])
        update = simplified[:, div_fc:div_co]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(simplified[:, div_co:])

        return output_gate * tf.tanh(state), state

      # Input data.
      train_data = list()
      for _ in range(num_unrollings + 1):
        train_data.append(
          tf.placeholder(tf.int64, shape=[batch_size]))
      train_inputs = train_data[:num_unrollings]
      train_labels = train_data[1:]  # labels are inputs shifted by one time step.

      # Unrolled LSTM loop.
      outputs = list()
      output = saved_output
      state = saved_state
      for i in train_inputs:
        output, state = lstm_cell(i, output, state, True)
        outputs.append(output)

      # State saving across unrollings.
      with tf.control_dependencies([saved_output.assign(output),
                                    saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

      # Optimizer.
      global_step = tf.Variable(0)
      learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate)
      gradients, v = zip(*optimizer.compute_gradients(loss))
      # gradient clipping
      gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
      optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

      # Predictions.
      train_prediction = tf.nn.softmax(logits)
      
      # Sampling and validation eval: batch 1, no unrolling.
      sample_input = tf.placeholder(tf.int64, shape=[1])
      saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
      saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
      reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
      sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
      with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                    saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


    num_steps = 7001
    summary_frequency = 100

    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print('Initialized')
      mean_loss = 0
      for step in range(num_steps):
        batches = bigram_train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
          feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run(
          [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
          if step > 0:
            mean_loss = mean_loss / summary_frequency
          # The mean loss is an estimate of the loss over the last few batches.
          print(
            'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
          mean_loss = 0
          labels = np.concatenate(list(batches)[1:])

          # convert from char ID to to one-hot encoding
          # and use those to calculate perplexity
          noembed_labels = np.zeros(predictions.shape)
          for i, j in enumerate(labels):
            noembed_labels[i, int(j)] = 1.0

          print('Minibatch perplexity: %.2f' % float(
            np.exp(logprob(predictions, noembed_labels))))
          
          if step % (summary_frequency * 10) == 0:
            # Generate some samples.
            print('=' * 80)
            for _ in range(5):

              # random start sample (in 1-hot encoding)
              feed = bigram_sample(bigram_random_distribution())

              # start sentence (in characters)
              # e.g. bigram_characters(feed) = ['ft']
              # convert from one-hot encoding to bigram ID
              
              bigram_sentence = bigram_characters(feed)

              # start sentence (in characters)
              # e.g. bigram_first_character(feed) = ['f']
              sentence = bigram_first_character(feed)[0]
              
              feed = [np.argmax(feed)]
              reset_sample_state.run()

              for _ in range(79):
                prediction = sample_prediction.eval({sample_input: feed})
                feed = bigram_sample(prediction)
                bigram_sentence += bigram_characters(feed)
                sentence += bigram_first_character(feed)[0]
                feed = [np.argmax(feed)]
              print('bigrams:', bigram_sentence)
              print('chars:', sentence)
            print('=' * 80)
          # Measure validation set perplexity.
          reset_sample_state.run()
          valid_logprob = 0
          for _ in range(valid_size):
            b = bigram_valid_batches.next()
            predictions = sample_prediction.eval({sample_input: [b[0][0]]})

            # convert char ID to one-hot encoding
            onehot_char = np.zeros((1, bigram_vocabulary_size))
            onehot_char[0, int(b[1][0])] = 1.0

            valid_logprob = valid_logprob + logprob(predictions, onehot_char)
          print('Validation set perplexity: %.2f' % float(np.exp(
            valid_logprob / valid_size)))
        if step == 7000:
          print("Loop {:d}, keep_rate {:f}".format(kr_loop_counter, kr))
          keep_rate_sum[kridx] += float(np.exp(valid_logprob / valid_size))
          print(keep_rate_sum)

          if kr == keep_rates[-1]:

            print("Dropout performance so far")
            for kr, krsum in zip(keep_rates, keep_rate_sum):
              print(kr, krsum/(kr_loop_counter+1))

loop 0
kr 0.95
Initialized
Average loss at step 0: 6.592824 learning rate: 10.000000
Minibatch perplexity: 729.84
bigrams: ['xi', 'bf', 'cr', 'wy', 'vb', 'bc', 'hq', 'ff', 'dy', 'ef', 'rv', 'wc', 'qa', 'pm', 'hh', 'qf', 'kj', 'cj', 'jy', 'hr', 'uj', 'rt', 'vp', 'hc', 'er', 'tw', 'eq', 'ef', 'wj', 'ty', 'wh', 'pq', 'sd', 'sp', 'sr', 'qv', 'vb', 'bd', 'es', 'ng', 'mp', 'iy', 'er', 'lv', 'mt', 'hg', 'rt', 'qa', 'ka', 'qr', 'ce', 'up', 'ji', 'lg', 'va', 'yo', 'qd', 'rv', 'th', 'ac', 'ai', 'fz', 'ny', 'wh', 'gi', ' w', 'dk', 'qn', 've', 'qb', 'uo', 'lh', 'fl', 'um', 'xi', 'yd', 'se', 'yu', 'bm', 'qt']
chars: xbcwvbhfderwqphqkcjhurvheteewtwpsssqvbenmielmhrqkqcujlvyqrtaafnwg dqvqulfuxysybq
bigrams: ['ko', 'xs', 'jb', 'lf', 'vp', 'an', 'by', 'ew', 'in', 'xz', 'mv', 'ow', 're', 'nl', 'zv', 'nn', 'oc', 'uy', 'wg', 'ik', 'os', 'ep', 'ui', 'yx', 'lg', ' m', 'ju', 'lt', 'tt', 'fm', 'jk', 'me', 'qx', 'zo', 'cu', 'cc', 'in', 'bi', 'kn', 'ai', ' y', 'tr', 'jx', 'wg', 'ra', 'kt', 'y ', 'am', 'il', 'jj'

More focused grid search

In [ ]:
keep_rates = [1, 0.9875, 0.975, 0.9625, 0.95, 0.9375, 0.925, 0.9125]
keep_rate_sum = [0.]*len(keep_rates)

for kr_loop_counter in xrange(10):

  for kridx, kr in enumerate(keep_rates):

    print("="*80 + "\n" + "="*80)
    print("loop", kr_loop_counter)
    print("kr", kr)

    graph = tf.Graph()
    with graph.as_default():

      # Parameters:
      # Input gate: input, previous output, and bias.
      simple_x = tf.Variable(tf.truncated_normal([bigram_vocabulary_size, 4*num_nodes], -0.1, 0.1))
      simple_m = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
      simple_b = tf.Variable(tf.zeros([1, 4*num_nodes]))
      # inpput, forget, memory, output range dividers
      div_if = num_nodes
      div_fc = num_nodes*2
      div_co = num_nodes*3
      # Variables saving state across unrollings.
      saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      # Classifier weights and biases.
      w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
      b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
      
      # Definition of the cell computation.
      def lstm_cell(i, o, state, train=False):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        # core of embedding approach
        if train:
          simplified = tf.nn.dropout(tf.nn.embedding_lookup(simple_x, i), kr) + tf.matmul(o, simple_m) + simple_b
        else:
          simplified = tf.nn.embedding_lookup(simple_x, i) + tf.matmul(o, simple_m) + simple_b
        input_gate = tf.sigmoid(simplified[:, :div_if])
        forget_gate = tf.sigmoid(simplified[:, div_if:div_fc])
        update = simplified[:, div_fc:div_co]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(simplified[:, div_co:])

        return output_gate * tf.tanh(state), state

      # Input data.
      train_data = list()
      for _ in range(num_unrollings + 1):
        train_data.append(
          tf.placeholder(tf.int64, shape=[batch_size]))
      train_inputs = train_data[:num_unrollings]
      train_labels = train_data[1:]  # labels are inputs shifted by one time step.

      # Unrolled LSTM loop.
      outputs = list()
      output = saved_output
      state = saved_state
      for i in train_inputs:
        output, state = lstm_cell(i, output, state, True)
        outputs.append(output)

      # State saving across unrollings.
      with tf.control_dependencies([saved_output.assign(output),
                                    saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

      # Optimizer.
      global_step = tf.Variable(0)
      learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate)
      gradients, v = zip(*optimizer.compute_gradients(loss))
      # gradient clipping
      gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
      optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

      # Predictions.
      train_prediction = tf.nn.softmax(logits)
      
      # Sampling and validation eval: batch 1, no unrolling.
      sample_input = tf.placeholder(tf.int64, shape=[1])
      saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
      saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
      reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
      sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
      with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                    saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


    num_steps = 7001
    summary_frequency = 100

    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print('Initialized')
      mean_loss = 0
      for step in range(num_steps):
        batches = bigram_train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
          feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run(
          [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
          if step > 0:
            mean_loss = mean_loss / summary_frequency
          # The mean loss is an estimate of the loss over the last few batches.
          print(
            'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
          mean_loss = 0
          labels = np.concatenate(list(batches)[1:])

          # convert from char ID to to one-hot encoding
          # and use those to calculate perplexity
          noembed_labels = np.zeros(predictions.shape)
          for i, j in enumerate(labels):
            noembed_labels[i, int(j)] = 1.0

          print('Minibatch perplexity: %.2f' % float(
            np.exp(logprob(predictions, noembed_labels))))
          
          if step % (summary_frequency * 10) == 0:
            # Generate some samples.
            print('=' * 80)
            for _ in range(5):

              # random start sample (in 1-hot encoding)
              feed = bigram_sample(bigram_random_distribution())

              # start sentence (in characters)
              # e.g. bigram_characters(feed) = ['ft']
              # convert from one-hot encoding to bigram ID
              
              bigram_sentence = bigram_characters(feed)

              # start sentence (in characters)
              # e.g. bigram_first_character(feed) = ['f']
              sentence = bigram_first_character(feed)[0]
              
              feed = [np.argmax(feed)]
              reset_sample_state.run()

              for _ in range(79):
                prediction = sample_prediction.eval({sample_input: feed})
                feed = bigram_sample(prediction)
                bigram_sentence += bigram_characters(feed)
                sentence += bigram_first_character(feed)[0]
                feed = [np.argmax(feed)]
              print('bigrams:', bigram_sentence)
              print('chars:', sentence)
            print('=' * 80)
          # Measure validation set perplexity.
          reset_sample_state.run()
          valid_logprob = 0
          for _ in range(valid_size):
            b = bigram_valid_batches.next()
            predictions = sample_prediction.eval({sample_input: [b[0][0]]})

            # convert char ID to one-hot encoding
            onehot_char = np.zeros((1, bigram_vocabulary_size))
            onehot_char[0, int(b[1][0])] = 1.0

            valid_logprob = valid_logprob + logprob(predictions, onehot_char)
          print('Validation set perplexity: %.2f' % float(np.exp(
            valid_logprob / valid_size)))
        if step == 7000:
          print("Loop {:d}, keep_rate {:f}".format(kr_loop_counter, kr))
          keep_rate_sum[kridx] += float(np.exp(valid_logprob / valid_size))
          print(keep_rate_sum)

          if kr == keep_rates[-1]:

            print("Dropout performance so far")
            for kr, krsum in zip(keep_rates, keep_rate_sum):
              print(kr, krsum/(kr_loop_counter+1))

loop 0
kr 1
Initialized
Average loss at step 0: 6.593880 learning rate: 10.000000
Minibatch perplexity: 730.61
bigrams: ['ux', 'jy', 'cl', 'yw', 'rz', 'gj', 'sm', 'ml', 'yg', 'qy', 'uz', 'mb', 'pl', 'fj', 'jq', 'ne', 'bz', 'ib', 'cn', 'ce', 'iv', 'ot', 'dg', 'x ', 'ex', 'cp', 'oz', 'uu', 'zg', 'ho', 'wz', 'bf', 'oj', 'il', 'lx', 'ob', 'ax', 'zm', 'lf', 'uv', 'qd', 'fz', ' o', 'tl', 'b ', 'kg', 'bo', 'ea', 'gk', 'ot', 'xz', 'xe', 'wl', 'fr', 'yg', 'dj', 'zq', 'ml', ' f', 're', 'pd', 'px', 'ae', 'qt', 'ah', 'ya', 'cl', 'yh', 'kw', 'ts', 'ov', 'lv', 'ob', 'pc', 'gd', 'rh', 'ha', 'dm', 'si', 'up']
chars: ujcyrgsmyqumpfjnbicciodxecouzhwboiloazluqf tbkbegoxxwfydzm rppaqaycyktolopgrhdsu
bigrams: ['vn', 'gz', 'vj', 'ek', 'fr', 'uq', 'ix', 'zx', 'pv', 'vs', 'yz', 'nr', 'ct', 'zi', 'ua', 'en', 'tt', 'bu', 'lg', 'rg', 'mj', 'pi', 'ix', 'rk', 'm ', 'l ', 'mm', 'dn', 'an', 'wc', 'pi', 'ar', 'zt', 'oo', 'px', ' c', 'ma', 'sd', 'tm', 'dy', 'lx', 'fz', 'xj', 'nl', 'rw', 'y ', 'nq', 'na', 'sq', 'vc', '

Best perplexity at 4.136 over 10 loops with dropout keep rate of 0.9875.

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

(on hold for now)